In [ ]:
#This Code has important parts here that I need to describe
#   1.  Imports
#   2.  Read Datasets from the csv file
#   3.  Split trian and test
#   4.  Tokenizing the input text
#   5.  Converting Labels to tensors
#   6.  Dataset Class
#   7.  Model Definition (Each Head has )
#   8.  Training Setup
#   9.  Training Loop
#   10. Saving the Model (In the Proper way in order to use it Later)
#   11. Evaluation / Testing
#   12. How to Use the Model Locally

In [2]:
# 1️⃣ Imports
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
from transformers import AutoModel, AutoTokenizer
import pandas as pd
from sklearn.model_selection import train_test_split

# 2️⃣ Read dataset
df = pd.read_csv("HEAR_Dataset.csv")  # adjust path if needed
list_of_texts = df["Review Text"].tolist()

# Labels (0=negative,1=neutral,2=positive,3=unmentioned)
labels_array = df[["Pricing", "Appointments", "Medical Staff", "Customer Service", "Emergency Services"]].values

# 3️⃣ Split train/test
texts_train, texts_test, labels_train, labels_test = train_test_split(
    list_of_texts, labels_array, test_size=0.2, random_state=42
)

# 4️⃣ Tokenizer
tokenizer = AutoTokenizer.from_pretrained("aubmindlab/bert-base-arabertv2")
train_encodings = tokenizer(texts_train, truncation=True, padding=True, return_tensors="pt")
test_encodings = tokenizer(texts_test, truncation=True, padding=True, return_tensors="pt")

# 5️⃣ Convert labels to tensor
labels_train_tensor = torch.tensor(labels_train, dtype=torch.long)  # note dtype long for CrossEntropy
labels_test_tensor = torch.tensor(labels_test, dtype=torch.long)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/611 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/384 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [3]:
# 6️⃣ Dataset class
class FeedbackDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item['labels'] = self.labels[idx]
        return item

train_dataset = FeedbackDataset(train_encodings, labels_train_tensor)
test_dataset = FeedbackDataset(test_encodings, labels_test_tensor)

In [4]:
# 7️⃣ Model definition
class MultiHeadClassifier(nn.Module):
    def __init__(self, bert_model_name="aubmindlab/bert-base-arabertv2", num_heads=5, num_classes=4):
        super().__init__()
        self.bert = AutoModel.from_pretrained(bert_model_name)
        hidden_size = self.bert.config.hidden_size
        self.heads = nn.ModuleList([nn.Linear(hidden_size, num_classes) for _ in range(num_heads)])

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        cls_embedding = outputs.last_hidden_state[:,0,:]
        logits = [head(cls_embedding) for head in self.heads]  # list of [batch, num_classes]
        return logits

In [5]:
model = MultiHeadClassifier(num_heads=5, num_classes=4)

# 8️⃣ Training setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
optimizer = optim.AdamW(model.parameters(), lr=2e-5)
criterion = nn.CrossEntropyLoss()
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)

model.safetensors:   0%|          | 0.00/543M [00:00<?, ?B/s]

In [ ]:
# 9️⃣ Training loop
for epoch in range(3):
    model.train()
    total_loss = 0
    for batch in train_loader:
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        outputs = model(input_ids, attention_mask)
        loss = 0
        for head_idx, head_logits in enumerate(outputs):
            loss += criterion(head_logits, labels[:, head_idx])

        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1} loss: {total_loss/len(train_loader)}")


In [ ]:
#   10 : Saving the Model
# 💾 SAVE AND DOWNLOAD TRAINED MODEL (Colab cell)

import torch
import os
from google.colab import files
import shutil

# 🧩 Create a main folder for the saved model
save_dir = "./saved_model"
os.makedirs(save_dir, exist_ok=True)

# 🧠 1️⃣ Save model weights
model_path = os.path.join(save_dir, "feedback_classifier.pt")
torch.save(model.state_dict(), model_path)
print(f"✅ Model weights saved at: {model_path}")

# 🔤 2️⃣ Save the tokenizer (very important for future use)
tokenizer.save_pretrained(os.path.join(save_dir, "tokenizer"))
print("✅ Tokenizer saved.")

# 📊 3️⃣ Save metrics (if you have the file from evaluation)
if os.path.exists("results/evaluation_results.txt"):
    shutil.copy("results/evaluation_results.txt", os.path.join(save_dir, "evaluation_results.txt"))
    print("✅ Evaluation results copied.")

# 📦 4️⃣ (Optional) Zip everything for easy download
zip_path = shutil.make_archive("saved_model", 'zip', save_dir)
print(f"📦 Model folder zipped at: {zip_path}")

# ⬇️ 5️⃣ Download the zip to your local computer
files.download(zip_path)
print("⬇️ Download started! Check your browser’s download bar.")

In [ ]:
# 1️⃣1️⃣ Evaluation / Testing
print("\n🚀 Starting evaluation...")

test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)
model.eval()

all_preds = [[] for _ in range(5)]
all_labels = [[] for _ in range(5)]
all_probs = [[] for _ in range(5)]

with torch.no_grad():
    for batch in test_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        outputs = model(input_ids, attention_mask)
        for i, logits in enumerate(outputs):
            probs = torch.softmax(logits, dim=1)
            preds = torch.argmax(probs, dim=1)

            all_preds[i].extend(preds.cpu().numpy())
            all_labels[i].extend(labels[:, i].cpu().numpy())
            all_probs[i].extend(probs.cpu().numpy())

aspect_names = ["Pricing", "Appointments", "Medical Staff", "Customer Service", "Emergency Services"]

metrics = []
for i, aspect in enumerate(aspect_names):
    y_true = np.array(all_labels[i])
    y_pred = np.array(all_preds[i])
    y_prob = np.array(all_probs[i])

    acc = accuracy_score(y_true, y_pred)
    prec = precision_score(y_true, y_pred, average='macro', zero_division=0)
    rec = recall_score(y_true, y_pred, average='macro', zero_division=0)
    f1 = f1_score(y_true, y_pred, average='macro', zero_division=0)

    # Compute mean Average Precision (mAP)
    y_true_onehot = np.zeros_like(y_prob)
    y_true_onehot[np.arange(len(y_true)), y_true] = 1
    try:
        mAP = average_precision_score(y_true_onehot, y_prob, average='macro')
    except ValueError:
        mAP = 0.0  # in case only one class appears

    metrics.append({
        "Aspect": aspect,
        "Accuracy": acc,
        "Precision": prec,
        "Recall": rec,
        "F1": f1,
        "mAP": mAP
    })

os.makedirs("results", exist_ok=True)
results_path = os.path.join("results", "evaluation_results.txt")
with open(results_path, "w", encoding="utf-8") as f:
    f.write("📊 Evaluation Results\n")
    f.write("=" * 50 + "\n")
    for m in metrics:
        f.write(
            f"{m['Aspect']}\n"
            f"  Accuracy: {m['Accuracy']:.4f}\n"
            f"  Precision: {m['Precision']:.4f}\n"
            f"  Recall: {m['Recall']:.4f}\n"
            f"  F1-score: {m['F1']:.4f}\n"
            f"  mAP: {m['mAP']:.4f}\n"
            + "-" * 50 + "\n"
        )

    mean_f1 = np.mean([m['F1'] for m in metrics])
    mean_map = np.mean([m['mAP'] for m in metrics])
    f.write(f"\nOverall Mean F1: {mean_f1:.4f}\n")
    f.write(f"Overall Mean mAP: {mean_map:.4f}\n")

print(f"✅ Evaluation completed. Results saved to: {results_path}")

files.download(results_path)
print("⬇️ Download started for evaluation_results.txt!")

In [ ]:
# 1️⃣2️⃣ Evaluation / Testing
#How to Use the Model Locally

import torch
from transformers import AutoTokenizer

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained("saved_model/tokenizer")

# Load model
model = MultiHeadClassifier()
model.load_state_dict(torch.load("saved_model/feedback_classifier.pt", map_location="cpu"))
model.eval()

print("✅ Model and tokenizer loaded successfully.")